This notebook stems from the starter code "bellsej.py"

In [1]:
# Some useful packages and helper functions

import sounddevice as sd
import numpy as np

Fs = 11025

def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

def load_data():
    mix = np.loadtxt('mix.dat')
    return mix

def play(vec):
    sd.play(vec, Fs, blocking=True)
    
def sigmoid(s):
    return 1 / (1 + np.exp(-s))

In [2]:
def unmixer(X):
    M, N = X.shape
    W = np.eye(N)

    anneal = [0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01,
              0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    ######## Your code here ##########
    
    # Use slowly decreasing learning rate for different epochs
    for learning_rate in anneal:
        
        for i in range(M):
            # Stochastic gradient ascent learning rule for ICA
            x_i = X[i, :]
            x_i = x_i.reshape(x_i.shape[0], 1)
            W += learning_rate * (np.dot((1 - 2 * sigmoid(np.dot(W, x_i))), x_i.T) + np.linalg.inv(W.T))
                
    ###################################
    return W

def unmix(X, W):
    S = np.zeros(X.shape)

    ######### Your code here ##########
    
    S = np.dot(W, X.T)
    S = S.T
    
    ##################################
    return S

In [3]:
X = normalize(load_data())
for i in range(X.shape[1]):
    print('Playing mixed track %d' % i)
    play(X[:, i])

Playing mixed track 0
Playing mixed track 1
Playing mixed track 2
Playing mixed track 3
Playing mixed track 4


In [4]:
W = unmixer(X)
S = normalize(unmix(X, W))

Separating tracks ...


In [5]:
for i in range(S.shape[1]):
    print('Playing separated track %d' % i)
    play(S[:, i])

Playing separated track 0
Playing separated track 1
Playing separated track 2
Playing separated track 3
Playing separated track 4


Different sources are now pretty clearly separated!